# Python pour biochimistes: soumettre des tâches dans une grappe de calcul via un script Python et un fichier de configuration

## Deuxième partie: Python, YAML et exécution

Ok, comment exploiter notre fichier de configuration dans notre programme Python? C'est facile car Python a un module capable de lire un fichier en format YAML pour construire automatiquement un dictionnaire. Avec ce dictionnaire, on pourra tout faire; il faut juste écrire le code nécessaire ;-) Dans l'exemple ci-dessous, le code est fortement commenté afin d'expliquer les tâches à faire. 

In [4]:
#####
# Exemple de programme pour créer les scripts nécessaires à l'exécution 
# des taches d'alignements HISAT2 sur une ensemble de fichiers FASTQ paired-end.
#
# Ce code n'est pas destiner à fonctionner tel quel!! Il est donné à titre
# démonstratif pour formation. 
# 
# NE PAS UTILISER SUR UNE GRAPPE SANS L'AVOIR ADAPTÉ À VOS DONNÉES ET VOTRE
# ENVIRONNEMENT!!
#####
#
# Modules nécessaires
#
import os
import re
import yaml
#
# Méthodes utiles
#
# 1 - Methode pour extraire les infos specifiques à SLURM pour la soumission 
#     des taches
#
# Ici, on regarde spécifiquement pour des paramètres spécifiques.
# Tout autre paramètre serait donc ignoré :-)
#
# uneListe provient de la lecture du fichier YAML
#
def slurmProcessor(uneListe):
  #
  # Première ligne nécessaire 
  #
  tmp = "#!/bin/bash -l\n";
  for unItem in uneListe:
      #
      # Voir les commentaires dans le fichier YAML de démonstration
      #
      if "account" in unItem:
          tmp = tmp+"#SBATCH --account="+unItem["account"]+" \n"
      elif "job-name" in unItem:
          tmp = tmp+"#SBATCH --job-name="+unItem["job-name"]+" \n"
      elif "mail-user" in unItem:
          tmp = tmp+"#SBATCH --mail-user="+unItem["mail-user"]+" \n"
      elif "mail-type" in unItem:
          tmp = tmp+"#SBATCH --mail-type="+unItem["mail-type"]+" \n"
      elif "time" in unItem:
          tmp = tmp+"#SBATCH --time="+unItem["time"]+" \n"
      elif "nodes" in unItem:
          tmp = tmp+"#SBATCH --nodes="+str(unItem["nodes"])+" \n"
      elif "cpus-per-task" in unItem:
          tmp = tmp+"#SBATCH --cpus-per-task="+str(unItem["cpus-per-task"]["val"])+" \n"
      elif "mem" in unItem:
          tmp = tmp+"#SBATCH --mem="+unItem["mem"]+" \n"
      
  #
  # On recupère la chaine de caractères en sortie 
  #
  return tmp;
#
# 2 - Methode pour extraire les infos specifiques à HISAT2
#     pour chaque paire de fichiers de séquences pour un 
#     échantillon donné
#
#     Une autre approche: comme chaque paramétre à soit comme valeur
#     un booleen ou une chaine de caractères, on parcours la liste
#     des parametres pour faire un test et écrire la chaine de caractères
#     correspondante. L'addition des valeurs pour -1, -2 et -S se fait 
#     après.
#
def hisatProcessor(uneListe):
  #
  # Ici, on assume que hisat2 est sur $PATH... Ajuster en conséquence :-)
  #
  tmp = "hisat2 "

  for i in uneListe:
      #
      # Cas spécial pour le paramétre x car on a un seul "-"
      #
      if "x" in i:
        keys = list(i)
        #
        # Il faut "escaper" le caractère \ pour le faire apparaitre
        # à la fin de chaque ligne.
        #
        # En faisant ça, on peut briser notre ligne de commande sans faire
        # planter le shell
        #
        tmp = tmp + "-"+keys[0]+" "+i["x"]+" \\\n"
      #
      # Cas spécial pour le paramétre threads car on doit extraire la
      # valeur dans un autre dictionnaire
      #
      elif "threads" in i:
        keys = list(i)
        par = keys[0]
        val = i["threads"].get("val")
        tmp = tmp + "--"+keys[0]+" "+str(val)+" \\\n"          
      #
      # Pour tous les autres paramètres spécifiés dans le fichier YAML. 
      #
      else:
        keys = list(i)
        par = keys[0]
        val = i[par]
        if val is True:
          tmp = tmp + "--"+par+" \\\n"
        else:
          tmp = tmp + "--"+par+" "+str(val)+" \\\n"
    
  #
  # On recupère la chaine de caractères en sortie 
  #
  return tmp;

#####################################################
#     Main -> Mettre en oeuvre notre automation     #
#####################################################
#
# Variable pour capturer la sortie des méthodes et des 
# operations sur fichiers.
# 
# À soumettre pour écrire les fichiers shell
#
aStr = ""
#
# Lire un parametre entré en ligne de commande sous sys.argv[1]. 
# FYI:sys.argv[0] est le nom du script
#
#with open(sys.argv[1], 'r') as stream:
#
# Mais pour simplifier cette démonstration, on lira directement 
# le fichier YAML...
#
with open("../z.misc_files/data_drm/hisat2_demo_test.yaml", 'r') as stream:
  #
  # On tente notre chance...
  #
  try:
    conf = yaml.safe_load(stream)
  #
  # ... Et si on a une erreur, voici ce que l'on va faire.
  #
  except yaml.YAMLError as uneError:
    print(uneError)
#
# Une façon d'aller chercher les infos via
# la methode get()
#
# Les methodes simplifient l'exécution
#
slurmHeader = slurmProcessor(conf.get("slurm_static"))
hisatParam = hisatProcessor(conf.get("hisat2_static"))
#
# Obtenir les PATHS pour lire les fichiers FASTQ et écrire les 
# fichiers de sortie SAM et log d'exécution.
#
folderIn = conf.get("in")
folderOut = conf.get("out")
#
# Ce sont les conditions qui amènent la répétition
#
for aCond in conf.get("conditions"):
    #
    # Ou sont les fichiers FASTQ?
    #
    samplesIn = folderIn+"/"+aCond
    #
    # Si le meme répertoire n'existe pas dans folderOut, on le crée
    #
    if os.path.exists(folderOut+"/"+aCond):
      print("Dossier existant!!")
    else:
      os.makedirs(folderOut+"/"+aCond)
      print("Dossier "+ aCond + "cree!!")
    files2align = os.listdir(samplesIn)
    #
    # Comme chaque echantillon produit deux fichiers, nous avons
    # forcément des doublons dans les noms de fichiers... 
    # Utiliser un ensemble (set) règle le problème ;-)
    #
    aList = []
    #
    # On parcours le contenu du repertoire
    #
    for aFile in files2align:
        #
        # En cas ou le repertoire contiendrait d'autres fichiers...
        # On teste pour la chaine de caractères qui défini les fichiers
        # FASTQ; en convertissant le résultat de la méthode search en 
        # booléen, ça retourne true pour les bons fichiers.
        #
        if bool(re.search(".R[1,2].fastqdemo",aFile)):
          aFileShort = re.sub(".R[1,2].fastqdemo","",aFile)
          aList.append(aFileShort)
    #
    # Conversion en ensemble pour retirer les doublons de noms de fichier!
    #
    aSet = set(aList)
    #
    # On tri selon l'ordre alphabétique :-)
    #    
    aSet = sorted(aSet)
    #
    # On a l'information nécessaire pour créer le fichier shell unique à chaque tache HISAT2
    #
    for sample in aSet:
      shellScriptPath = folderOut+"/"+aCond+"/"+sample+".sh"
      shellScript = open(shellScriptPath,"w")
      shellOutput = "#SBATCH --output="+os.path.abspath(folderOut+"/"+aCond+"/"+sample)+".log\n"
      slurmHeader = slurmHeader+shellOutput
      shellScript.write(slurmHeader)
      #
      # La méthode os.path.abspath est une sécurité pour s'assurer que le script fonctionnera
      # L'utilisation des parcours relatifs peut etre risqué...
      #
      firstRead =  os.path.abspath(folderIn+"/"+aCond+"/"+sample+".R1.fastqdemo")
      secondRead = os.path.abspath(folderIn+"/"+aCond+"/"+sample+".R2.fastqdemo")
      samOutput = os.path.abspath(folderOut+"/"+aCond+"/"+sample+".sam")

      hisatParam = hisatParam+"-1 "+firstRead+" \\\n"
      hisatParam = hisatParam+"-2 "+secondRead+" \\\n"
      hisatParam = hisatParam+"-S "+samOutput
    
      shellScript.write(hisatParam)
      shellScript.close()
      #
      # Si dry-run est à false, le script sera soumis 
      #
      if conf.get("dry-run")==False:
          os.system("sbatch "+shellScriptPath)
          print("Script "+shellScriptPath+" soumis pour execution")

Dossier existant!!
Dossier existant!!
Dossier existant!!
Dossier existant!!
